# Bounding Box Regression
[PyImageSearch | Object detection: Bounding box regression with Keras, TensorFlow, and Deep Learning](https://pyimagesearch.com/2020/10/05/object-detection-bounding-box-regression-with-keras-tensorflow-and-deep-learning/)

## Reading Data

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# define the base path to the input dataset and then use it to derive
# the path to the images directory and annotation CSV file
BASE_PATH = "/content/drive/MyDrive/ML_Project/Data/BoundingBoxTraining"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "annotations.json"])

# define the path to the base output directory
BASE_OUTPUT = "/content/output"
# define the path to the output serialized model, model training plot,
# and testing image filenames
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_FILENAMES = os.path.sep.join([BASE_OUTPUT, "test_images.txt"])

# initialize our initial learning rate, number of epochs to train
# for, and the batch size
INIT_LR = 4e-4
NUM_EPOCHS = 30
BATCH_SIZE = 16

In [ ]:
# load the contents of the CSV annotations file
print("[INFO] loading dataset...")
json_data = json.load(open(ANNOTS_PATH))
# initialize the list of data (images), our target output predictions
# (bounding box coordinates), along with the filenames of the
# individual images
data = []
targets = []
filenames = []

In [ ]:

'''
Multiple annotations are present in the diff row
'''
for key, value in list(json_data.items()):
  filename = key
  bboxes = []
  imagePath = os.path.sep.join([IMAGES_PATH, filename])
  if not os.path.exists(imagePath):
    break
  # print(imagePath)
  image = cv2.imread(imagePath)
  (h, w) = image.shape[:2]
  # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image,(224, 224))
  # image = cv2.erode(image, None, iterations=1)
  image = img_to_array(image)
  for ann in value['ann']:
    if ann['cls'] == 'date':
      startX, startY, endX, endY = ann['bbox']
      startX = float(startX) / w
      startY = float(startY) / h
      endX = float(endX) / w
      endY = float(endY) / h
      print(startX, startY, endX, endY)
      bboxes.append([startX, startY, endX, endY])
      data.append(image)
      targets.append((startX, startY, endX, endY))
      filenames.append(filename)

In [ ]:
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
split = train_test_split(data, targets, filenames, test_size=0.10,
	random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
f = open(TEST_FILENAMES, "w")
f.write("\n".join(testFilenames))
f.close()

## Training

In [ ]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(256, activation="relu")(flatten)
# bboxHead = Dropout(0.5)(bboxHead)
# //bboxHead = Dense(256, activation="relu")(bboxHead)
# bboxHead = Dropout(0.5)(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
# bboxHead = Dropout(0.5)(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
# bboxHead = Dropout(0.5)(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
# bboxHead = Dropout(0.5)(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

In [ ]:
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt, metrics=[
        'MeanSquaredError',
        'accuracy',
    ])
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(
	trainImages, trainTargets,
	validation_data=(testImages, testTargets),
	batch_size=BATCH_SIZE,
	epochs=NUM_EPOCHS,
	verbose=1)

In [ ]:
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(MODEL_PATH, save_format="h5")
# plot the model training history
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)

## Testing

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes
import argparse
import imutils
import cv2
import os

In [ ]:
# test_image = '/content/drive/MyDrive/ML_Project/Data/BoundingBoxTraining/images/img_00010.jpg'
test_image = '/content/output/test_images.txt'

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", required=False,
	help="path to input image/text file of image filenames", default=test_image)
# args = vars(ap.parse_args())
args = ap.parse_args(args=[])
print(args.input)

In [ ]:
# determine the input file type, but assume that we're working with
# single input image
filetype = mimetypes.guess_type(args.input)[0]
print(filetype)
imagePaths = [args.input]
# if the file type is a text file, then we need to process *multiple*
# images
if "text/plain" == filetype:
	# load the filenames in our testing file and initialize our list
	# of image paths
	filenames = open(args.input).read().strip().split("\n")
	imagePaths = []
	# loop over the filenames
	for f in filenames:
		# construct the full path to the image filename and then
		# update our image paths list
		p = os.path.sep.join([IMAGES_PATH, f])
		imagePaths.append(p)
print(imagePaths)

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
print("[INFO] loading object detector...")
model = load_model(MODEL_PATH)
# loop over the images that we'll be testing using our bounding box
# regression model
for imagePath in imagePaths:
	# load the input image (in Keras format) from disk and preprocess
	# it, scaling the pixel intensities to the range [0, 1]
  image = cv2.imread(imagePath)
  (h, w) = image.shape[:2]
  # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image,(224, 224))
  # image = cv2.erode(image, None, iterations=1)
  image = img_to_array(image) / 255.0
  image = np.expand_dims(image, axis=0)
  # make bounding box predictions on the input image
  preds = model.predict(image)[0]
  (startX, startY, endX, endY) = preds
  # load the input image (in OpenCV format), resize it such that it
  # fits on our screen, and grab its dimensions
  image = cv2.imread(imagePath)
  image = imutils.resize(image, width=600)
  (h, w) = image.shape[:2]
  print(h,w)
  # scale the predicted bounding box coordinates based on the image
  # dimensions
  startX = int(startX * w)
  startY = int(startY * h)
  endX = int(endX * w)
  endY = int(endY * h)
  # show the output image
  cv2_imshow(image)
  # show the output image
  print(image.shape)
  bb = image[startY-10:endY+10, startX-10: endX+10]
  # cv2.imwrite('/content/output/test.png', bb)
  cv2_imshow(bb)

# Data Information

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow 
import os
import re
import numpy as np
import keras

In [ ]:
img_path = '/content/img_00001.jpg'
img = cv2.imread(img_path)
cv2_imshow(img)

In [ ]:
annotation_path = '/content/annotations.json'

In [ ]:
import json
  
def show_image_with_bounding_box(image_path, annotation_path):
  img = cv2.imread(image_path)
  json_data = json.load(open(annotation_path))
  filename = image_path.split('/')[-1]
  data = json_data[filename]['ann']
  # plt.imshow(img)
  for a in data:
    bbox = a['bbox']
    cls = a['cls']
    x1 = int(bbox[0])
    y1 = int(bbox[1])
    x2 = int(bbox[2])
    y2 = int(bbox[3])
    if cls == 'date':
      cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0), 2)




 
  scale_percent = 220 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  plt.figure(figsize=(10, 10))
  plt.imshow(resized)

In [ ]:
ROOT = '/content/images'
all_images = []
for path in os.listdir(ROOT):
  if os.path.isfile(os.path.join(ROOT, path)):
    all_images.append(os.path.join(ROOT, path))
print(all_images)

In [ ]:
for img_path in all_images:
  show_image_with_bounding_box(img_path, annotation_path)

In [ ]:
/content/output/detector.h5

In [ ]:
!pip install easyocr